# Collaborative filtering
Possiamo scaricare i dataset da Kaggle utilizzando la libreria opendatasets.
Il metodo od.download consente di scaricare un dataset utilizzando l'url della pagina di Kaggle.

Viene richiesto di inserire il proprio username e un'API key.

L'API key può essere generata andando nel proprio profilo (icona in alto a destra della pagina), cliccando poi su settings e infine su create new token.

Scaricherà un file json che può essere aperto con blocco note contenente le credenziali.

In [1]:
!pip install opendatasets -q

import opendatasets as od
od.download("https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: lucagagliardelli
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system


100%|██████████| 165M/165M [00:00<00:00, 1.61GB/s]

Avviamo la sessione di Spark

In [1]:
# Installa le due librerie
!pip install pyspark -q
!pip install findspark -q

# Importa spark e findspark
import pyspark
import findspark
findspark.init()

# Definisce l'sqlContext
from pyspark.sql import SparkSession

# Imposta la memoria utilizzabile da Spark a 12GB che è il massimo disponibile in Colab
sqlContext = SparkSession.builder\
            .config("spark.driver.memory", "2g")\
            .config("spark.executor.memory", "5g")\
            .config("spark.executor.instances", "2")\
            .config("spark.kryoserializer.buffer.max", "128m").getOrCreate()

Importiamo le librerie necessarie

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer

Carichiamo i dati scaricati da Kaggle.

In [3]:
ratings = sqlContext.read.option("header", True).option("inferSchema", True).csv("/content/movie-recommendation-system/ratings.csv")
ratings.show(10)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
+------+-------+------+----------+
only showing top 10 rows



Visualizziamo lo schema e il numero di record

In [6]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
ratings.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|         25000095|          25000095|          25000095|            25000095|
|   mean|81189.28115381162|21387.981943268616| 3.533854451353085|1.2156014431215513E9|
| stddev|46791.71589745776| 39198.86210105973|1.0607439611423535| 2.268758080595386E8|
|    min|                1|                 1|               0.5|           789652009|
|    max|           162541|            209171|               5.0|          1574327703|
+-------+-----------------+------------------+------------------+--------------------+



In [5]:
ratings.count()

25000095

I dati sono troppi per essere gestiti su Google Colab.
Teniamo i dati solo dei primi 150 utenti giusto per fare un'esempio.

In [ ]:
ratings = ratings.where(ratings.userId <= 150)

ratings.count()

18262

Ora dividiamo i dati in training e test e alleniamo il modello

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId",
          ratingCol="rating",
          coldStartStrategy="drop")

model = als.fit(training)

Valutiamo il modello sui dati di test e vediamo l'errore quadratico medio.

Non è bassissimo, bisogna considerare che i voti vanno da 0 a 5 e l'errore medio è 1.7

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.7461987538672745


Ora è possibile fare varie predizioni utilizzando il modello.

Si può prevedere il miglior film da consigliare ad ogni utente. Tra tutti quelli che l'utente non ha giudicato, ritorna i primi N (sulla base del valore fornito) con il voto calcolato più alto.

Modello user-based.

In [ ]:
# Il parametro dice il numero di film da ritornare
userRecs = model.recommendForAllUsers(1)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{72226, 6.4924927}]|
|     2| [{8784, 6.3909955}]|
|     3|  [{3421, 6.092199}]|
|     4|  [{1407, 5.701938}]|
|     5|   [{30749, 6.2793}]|
|     6| [{1278, 6.7345967}]|
|     7|[{31658, 6.5279164}]|
|     8|  [{1243, 6.987815}]|
|     9|[{115569, 6.480059}]|
|    10| [{3421, 6.6807785}]|
|    11|  [{2804, 8.241351}]|
|    12| [{2572, 5.9282928}]|
|    13| [{2019, 5.9035463}]|
|    14|   [{265, 8.485392}]|
|    15| [{6016, 7.2264376}]|
|    16|  [{912, 6.5533547}]|
|    17|  [{509, 5.2396007}]|
|    18| [{81591, 5.676386}]|
|    19|  [{246, 6.5038114}]|
|    20|[{122904, 6.820263}]|
+------+--------------------+
only showing top 20 rows



Oppure per ogni film scegliere l'utente a cui consigliarlo.

Modello item-based.

In [ ]:
# Il parametro dice il numero di utenti da ritornare
movieRecs = model.recommendForAllItems(1)
movieRecs.show()

+-------+------------------+
|movieId|   recommendations|
+-------+------------------+
|      1|  [{49, 7.003292}]|
|      3|  [{56, 8.155155}]|
|      5|  [{22, 6.075884}]|
|      6| [{143, 5.986134}]|
|      9|  [{54, 4.955309}]|
|     16| [{32, 6.6986012}]|
|     17|  [{22, 7.629116}]|
|     19|  [{56, 5.877934}]|
|     20|[{127, 1.1382544}]|
|     22| [{127, 5.213947}]|
|     26| [{32, 3.7949572}]|
|     27|  [{8, 3.1816194}]|
|     28| [{127, 7.084346}]|
|     31|  [{56, 6.426809}]|
|     34| [{49, 7.3857627}]|
|     35| [{56, 4.1258173}]|
|     40| [{54, 4.4710474}]|
|     44|  [{54, 4.625162}]|
|     47| [{129, 7.248899}]|
|     48| [{135, 6.855863}]|
+-------+------------------+
only showing top 20 rows



è possibile fare predizioni per utenti specifici.

Ad esempio, consigliamo 10 film all'utente con id=1

In [ ]:
users = ratings.select(als.getUserCol()).where(ratings.userId == 1)
userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{72226, 6.492492...|
+------+--------------------+



Stessa cosa può essere fatta con i film

In [ ]:
movies = ratings.select(als.getItemCol()).where(ratings.movieId == 1)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|      1|[{49, 7.003292}, ...|
+-------+--------------------+



Possiamo visualizzare i titoli dei film consigliati.

In [ ]:
from pyspark.sql.functions import explode, col

hints = userSubsetRecs\
.select(userSubsetRecs.userId, explode(userSubsetRecs.recommendations).alias('rec'))\
.select(col('userId'), col("rec.movieId"), col("rec.rating"))

movies = sqlContext.read.option("header", True).option("inferSchema", True).csv("/content/movie-recommendation-system/movies.csv")

join = hints.join(movies, on="movieId")
join.show(truncate=False)

+-------+------+---------+--------------------------------------------------+--------------------------------------------------+
|movieId|userId|rating   |title                                             |genres                                            |
+-------+------+---------+--------------------------------------------------+--------------------------------------------------+
|72226  |1     |6.4924927|Fantastic Mr. Fox (2009)                          |Adventure|Animation|Children|Comedy|Crime         |
|31658  |1     |6.2336073|Howl's Moving Castle (Hauru no ugoku shiro) (2004)|Adventure|Animation|Fantasy|Romance               |
|109487 |1     |5.75041  |Interstellar (2014)                               |Sci-Fi|IMAX                                       |
|741    |1     |5.623687 |Ghost in the Shell (Kôkaku kidôtai) (1995)        |Animation|Sci-Fi                                  |
|108932 |1     |5.619531 |The Lego Movie (2014)                             |Action|Adventure|Ani